# Training Materials: Detecting Change with 2-Time Period Data

## 1. Introduction: Change Detection Basics

* **What is Change Detection?** Change detection is the process of identifying differences in the state of an object or phenomenon by observing it at different times.  Its purposes include monitoring land cover changes, assessing damage from natural disasters, and tracking urban development. Applications range from deforestation monitoring and urban sprawl analysis to disaster assessment and agricultural monitoring.

* **Types of Change Detection:** Several approaches exist, including image differencing (subtracting pixel values), image ratioing (dividing pixel values), and classification-based methods (comparing land cover classifications).

* **Two-Time Period Analysis:** This training focuses on analyzing change between two specific points in time. Advantages include simplicity and clear visualization of change. Limitations include potential misinterpretation due to seasonal variations or short-term events.

## 2. Change Detection Methods
### Change detection analysis based on analysis of 2-time period data of ALOS-2/Palsa-2

In this section,By analyzing the HH/HV scattering intensities during the two periods with Change Vector Analysis, the procedure for detecting and analyzing changes will be practiced. In the previous section, we analyzed land cover and land use changes by comparing color tones and patterns in color composite images from year to year.

Change vector analysisis a method used to analyze changes in 2D data points and is an effective method for analyzing HH/HV changes in PALSAR-2. In this section, we will deepen our understanding of the procedures and interpretations by analyzing the 2007 and 2010 annual composite data.

1. Conversion from DN values to backscatter coefficients
2. HH/HV change analysis
3. Change vector analysis (CVA) Data analysis


### 2.1. Conversion from DN values to backscatter coefficients

In the previous section, the pixel value（DN; digital number）was used for visualization as is, but for rigorous data analysis, it is desirable to convert the pixel values ​​into backscattering coefficients. The conversion is done using the following formula:

$$ \gamma^{0} = 10 log_{10} <DN^2> + CF $$

Note that, <> represents averaging for noise reduction and -83.0 dB is used for CF. [Global 25m Resolution PALSAR-2/PALSAR Mosaic (Ver. 2.4.0) Dataset Instructions](https://www.eorc.jaxa.jp/ALOS/jp/dataset/pdf/DatasetDescription_PALSAR2_Mosaic_ver240_ja.pdf). A coding example is shown below.

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

def calibrate_ima(image):
    return 10 * np.log10(image ** 2) - 83.0

def ima_cal(file_path):
    with rasterio.open(file_path) as src:
        original = src.read(1).astype('float32')
        calibrated = calibrate_ima(original)
    return original, calibrated

# Simplified file names
image1 = "image1.tif"  # Represents Alos_2_2007_HH
image2 = "image2.tif"  # Represents Alos_2_2010_HH
image3 = "image3.tif"  # Represents Alos_2_2007_HV
image4 = "image4.tif"  # Represents Alos_2_2010_HV

# Process images
image1_raw, image1_cal = ima_cal(image1)
image2_raw, image2_cal = ima_cal(image2)
image3_raw, image3_cal = ima_cal(image3)
image4_raw, image4_cal = ima_cal(image4)

# Plot results
fig, axs = plt.subplots(4, 2, figsize=(12, 16))

axs[0, 0].imshow(image1_raw, cmap='gray')
axs[0, 0].set_title('Image 1 (Original)')
axs[0, 1].imshow(image1_cal, cmap='gray')
axs[0, 1].set_title('Image 1 (Calibrated)')

axs[1, 0].imshow(image2_raw, cmap='gray')
axs[1, 0].set_title('Image 2 (Original)')
axs[1, 1].imshow(image2_cal, cmap='gray')
axs[1, 1].set_title('Image 2 (Calibrated)')

axs[2, 0].imshow(image3_raw, cmap='gray')
axs[2, 0].set_title('Image 3 (Original)')
axs[2, 1].imshow(image3_cal, cmap='gray')
axs[2, 1].set_title('Image 3 (Calibrated)')

axs[3, 0].imshow(image4_raw, cmap='gray')
axs[3, 0].set_title('Image 4 (Original)')
axs[3, 1].imshow(image4_cal, cmap='gray')
axs[3, 1].set_title('Image 4 (Calibrated)')

# Add colorbars
for i in range(4):
    fig.colorbar(axs[i, 0].imshow(image1_raw, cmap='gray'), ax=axs[i, 0], orientation='vertical', label='DN')
    fig.colorbar(axs[i, 1].imshow(image1_cal, cmap='gray'), ax=axs[i, 1], orientation='vertical', label='Calibrated dB')

plt.tight_layout()
plt.show()


### 2.3. HH/HV change analysis

 Calculate the difference in image data to analyze the change from 2007 to 2010 for each HH/HV.

In [ ]:
delta_bandhv_cal = hv_2010_cal - hv_2007_cal
delta_bandhh_cal = hh_2010_cal - hh_2007_cal

fig, axs = plt.subplots(1, 2, figsize=(12, 16))

axs[0].imshow(delta_bandhv_cal, cmap='gray')
axs[0].set_title('HV change (2010 - 2007)')
axs[1].imshow(delta_bandhh_cal, cmap='gray')
axs[1].set_title('HH change (2010 - 2007)')
plt.tight_layout()
plt.show()

### 2.4. Change vector analysis (CVA) Data analysis

The changes that appear in the difference image calculated in the previous section can be expressed as vectors in the two-dimensional space of HH/HV pairs for each pixel.The figure below shows an example where HV has decreased significantly while HH has decreased slightly. The following code cell calculates the length of the change vector at each pixel, expressed as magnitude, and the direction as angle.

In [ ]:
magnitude = np.sqrt(delta_bandhv_cal**2 + delta_bandhh_cal**2)
angle = np.arctan2(delta_bandhv_cal, delta_bandhh_cal) #Radians: When using trigonometric functions like arctan or atan2, the default output is often in radians, where 00 to 2𝜋2π radians (approximately 00 to 6.286.28) covers the full circular range of angles.